# Imports

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from matplotlib.pyplot import twinx
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Functions Library

In [2]:
def read_data() -> tuple[pd.Series, pd.Series]:
    PATH = r"C:\Users\pcampos\OneDrive - Insper - Instituto de Ensino e Pesquisa\Dissertação Mestrado\base\BR CDS and FX.xlsx"
    sheet_name = "BRL"
    sheet_name2 = "Brazil CDS"
    fx = pd.read_excel(PATH, index_col=0, sheet_name=sheet_name).iloc[:, 0]
    cds = pd.read_excel(PATH, index_col=0, sheet_name=sheet_name2).iloc[:, 0]
    fx.index = pd.to_datetime(fx.index)
    fx.name = sheet_name
    cds.index = pd.to_datetime(cds.index)
    cds.name = sheet_name2
    return fx, cds


def filter_by_period(df: pd.DataFrame, period: str) -> pd.DataFrame:
    new_index = (
        pd.DatetimeIndex(  # TODO: add check for last date, if should be included or not
            df.index.to_series().groupby(df.index.to_period(period)).max()
        )
    )
    return df.loc[new_index].copy()


def _calculate_parameters(y: pd.Series, x: pd.Series) -> pd.Series:
    model = sm.OLS(y, sm.add_constant(x)).fit()
    return model.params

def _percentage_formatter(x, _):
    return f"{x * 100:.1f}%"

# Prepare data

## Generate Return Series 

In [6]:
(s_index_fx, s_index_cds) = read_data()

list_series = [s_index_cds, s_index_fx]
df_indexes = pd.concat(list_series, axis=1, join="outer").fillna(method="ffill")
df_indexes.index.name = None

df_period = filter_by_period(df_indexes, "D")
df_return_ln = np.log(df_period / df_period.shift(1)).dropna().copy()
df_return = (df_period / df_period.shift(1) - 1).dropna().copy()
df_return_ln

,Brazil CDS,BRL
2007-08-08,0.004515,0.012847
2007-08-09,-0.006302,-0.013849
2007-08-10,-0.001671,-0.023582
2007-08-13,0.003102,0.008677
2007-08-14,-0.004417,-0.018366
...,...,...
2024-05-20,0.000455,0.004086
2024-05-21,-0.000379,-0.003153
2024-05-22,-0.001365,-0.008490
2024-05-23,-0.001671,0.000464


## Estimate Parameters

In [11]:
# Parameters:
# starting point: 252 observations
# expaning window beta

N_MIN = 252

N_MAX = len(df_return_ln.index)

aux_params = {}

for n in range(N_MIN, N_MAX + 1):

    sub_df = df_return_ln.iloc[:n].copy()

    param = _calculate_parameters(sub_df.iloc[:, 0], sub_df.iloc[:, 1])

    ref_date = sub_df.index[-1]
    aux_params[ref_date] = param.to_dict()


df_params = pd.DataFrame(aux_params).T
df_params.columns = ["alpha", "beta"]
df_aux = pd.concat([df_return_ln, df_params], axis=1)

# Trading Strategy

In [ ]:
df_trading_period = df_aux.dropna().copy()
N_DAYS = 63
rebalance_dates = df_trading_period.index[::N_DAYS].copy()

start_date = df_trading_period.index[0]
_beta = df_trading_period.loc[start_date, "beta"]
_period_return_acc_fx = 0
_period_return_acc_cds = 0
_return_acc_gap = 0
s_period_returns_acc_fx = pd.Series(name="period_returns_acc_fx")
s_period_returns_acc_cds = pd.Series(name="period_returns_acc_cds")
s_period_returns_acc_fx_expected = pd.Series(name="period_returns_acc_fx_expected")
s_period_returns_acc_gap = pd.Series(name="period_returns_acc_gap")
s_returns_acc_gap = pd.Series(name="returns_acc_gap")
s_period_nbr = pd.Series(name="period_nbr")
period = 1
for date in df_trading_period.index:
    s_period_nbr[date] = period
    _daily_return_fx = df_trading_period.loc[date, "BRL"]
    _period_return_acc_fx += _daily_return_fx
    s_period_returns_acc_fx[date] = _period_return_acc_fx

    _daily_return_cds = df_trading_period.loc[date, "Brazil CDS"]
    _period_return_acc_cds += _daily_return_cds
    s_period_returns_acc_cds[date] = _period_return_acc_cds

    _period_return_acc_fx_expected = _beta * _period_return_acc_fx
    s_period_returns_acc_fx_expected[date] = _period_return_acc_fx_expected
    _period_return_acc_gap = _period_return_acc_fx - _period_return_acc_fx_expected
    s_period_returns_acc_gap[date] = _period_return_acc_gap
    s_returns_acc_gap[date] = _period_return_acc_gap + _return_acc_gap

    rebalance_date = date in rebalance_dates
    if rebalance_date and date != start_date:
        _beta = df_trading_period.loc[date, "beta"]
        # _period_factor_acc_fx = 1
        # _period_factor_acc_cds = 1
        _period_return_acc_fx = 0
        _period_return_acc_cds = 0
        period += 1
        _signal = 1 if _period_return_acc_gap >= 0 else -1
        _return_acc_gap = (1 + _period_return_acc_gap) * (1 + _return_acc_gap) - 1

s_returns_acc_gap.plot()

In [ ]:
s_returns_acc_gap

In [ ]:
s_returns_acc_gap.plot()

In [ ]:
df_results_period = pd.concat(
    [
        s_period_returns_acc_fx,
        s_period_returns_acc_cds,
        s_period_returns_acc_fx_expected,
        s_period_returns_acc_gap,
    ],
    axis=1,
)
(1 + s_period_returns_acc_gap)

# Analysis

## Plot regression parameters

In [ ]:
# plot return
fig, axes = plt.subplots(figsize=(18, 12), nrows=2)

axes[0].set_title("BRL x Brazil CDS")
((1 + df_aux.iloc[:, :2]).cumprod() - 1).plot(ax=axes[0], color=["blue", "orange"])
axes[0].legend()

# Add gridlines to the first plot
axes[0].grid(True, color="gray", linestyle="--")

# Add a horizontal line at y=0 to the first plot
axes[0].axhline(y=0, color="black")

# Move the y-axis label and ticks to the right for the first plot
axes[0].yaxis.set_label_position("right")
axes[0].yaxis.tick_right()
axes[0].set_ylabel("Cumulative Log Returns")

# Apply the percentage formatter to the first plot
axes[0].yaxis.set_major_formatter(FuncFormatter(_percentage_formatter))

# Set x-axis limits to the first and last data points
axes[0].set_xlim(df_aux.index.min(), df_aux.index.max())

# beta and alpha plot
df_aux["beta"].plot(ax=axes[1], color="blue", label="Beta")
ax2 = axes[1].twinx()
df_aux["alpha"].plot(ax=ax2, color="orange", label="Alpha")

# Add gridlines to the second plot
axes[1].grid(True, color="gray", linestyle="--")

# Set y-axis labels
axes[1].set_ylabel("Beta")
ax2.set_ylabel("Alpha")

# Set x-axis limits to the first and last data points
axes[1].set_xlim(df_aux.index.min(), df_aux.index.max())

y_min, y_max = axes[1].get_ylim()
y_range = y_max - y_min
y_mid = (y_max + y_min)/2
axes[1].set_ylim(y_mid - y_range / 2, y_mid + y_range / 2 * 1.2)

y_min, y_max = ax2.get_ylim()
y_range = y_max - y_min
y_mid = (y_max + y_min) / 2
ax2.set_ylim(y_mid - y_range / 2, y_mid + y_range / 2 * 1.2)

# Combine legends from both axes
lines1, labels1 = axes[1].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
axes[1].legend(lines1 + lines2, labels1 + labels2, loc="upper left", ncol=2)

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust the layout to make space for the title

plt.show()
plt.close()